In [1]:
import os

In [2]:
%pwd

'd:\\Satellite-Image-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Satellite-Image-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch
import time

In [9]:
!pip install tensorboardX

In [10]:
import os
import time
from tensorboardX import SummaryWriter
import torch

class PrepareCallback:
    def __init__(self, config):
        self.config = config
        self.tb_writer = None

    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return SummaryWriter(logdir=tb_running_log_dir)

    def _create_ckpt_callbacks(self):
        return torch.save(self.config.model.state_dict(), self.config.checkpoint_model_filepath)

    def get_tb_ckpt_callbacks(self):
        self.tb_writer = self._create_tb_callbacks()
        return [
            self._create_ckpt_callbacks
        ]


In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

config\config.yaml
[2023-09-04 02:40:01,257: INFO: common: yaml file: config\config.yaml loaded successfully]
params.yaml
[2023-09-04 02:40:01,262: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-04 02:40:01,264: INFO: common: created directory at: artifacts]
[2023-09-04 02:40:01,267: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-09-04 02:40:01,269: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [12]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-09-04-02-40-58'